# Convert mw NetCDF to Tif

In [1]:
import numpy as np
import xarray
import rasterio
import pandas as pd

In [2]:
mw_path = r"../Data/microwave-rs/CumJour-North-ssmi-1979-2022-H19.nc"
mw_out_path = r'../Data/microwave-rs/mw-tif/'

In [3]:
dataset_mw = xarray.open_dataset(mw_path).rio.write_crs(
        "epsg:3413", inplace=True)

In [4]:
datelist = pd.date_range(start="2019-07-01",end="2019-07-31")

In [5]:
for i in datelist:
    #print(i) # date
    dataset_day = dataset_mw.where(dataset_mw["time"] == i, drop=True)
    data_mw_day = dataset_day["melt"]
    out_path_and_name = f"{mw_out_path}{i.date()}_mw.tif"
    data_mw_day.rio.to_raster(out_path_and_name)

    with rasterio.open(out_path_and_name) as src_mw:
        mw = src_mw.read()
        np.nan_to_num(mw, nan=-1, copy= False) # inplace
        kwargs1 = src_mw.meta.copy()

    # overwrite mw with new band (fixed nan)
    with rasterio.open(out_path_and_name, 'w', **kwargs1) as dst:
        dst.write_band(1, mw[0])

